In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
import plotly.graph_objects as go

from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [3]:
ogra = Ograyspy()
print(ogra.info_node)
to_be_found = 'Genie_Transfer'
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra

Found folder name:  C:\Users\mmaduar\.atom
Partes:  ('C:\\', 'Users', 'mmaduar', '.atom')
No. spec files:  0
I56611

Exec ogra.define_files_folder(to_be_found):
Found folder name:  C:\Users\mmaduar\Nextcloud\Genie_Transfer
Partes:  ('C:\\', 'Users', 'mmaduar', 'Nextcloud', 'Genie_Transfer')
No. spec files:  6891
WindowsI56611.pkl


In [4]:
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Windows-10-10.0.19045-SP0,AMD64,Windows,I56611,C:\Users\mmaduar,C:\Users\mmaduar\Nextcloud\Genie_Transfer,6891,0,,,[C:\Users\mmaduar\Nextcloud\Genie_Transfer\BGN...,"[BGNOVO.Chn, 2018/CCI/CCI (I-123) 210218.Chn, ...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",WindowsI56611.pkl


In [5]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [6]:
reduc_nms

['BGNOVO.Chn',
 '2018/CCI/CCI (I-123) 210218.Chn',
 '2018/CCI/CCI0404F.Chn',
 '2018/CCI/CCI0404I.Chn',
 '2018/CCI/CCI0703F.Chn',
 '2018/CCI/CCI0703I.Chn',
 '2018/CCI/CCI0808I.Chn',
 '2018/CCI/CCI0905 I-123.Chn',
 '2018/CCI/CCI0905F.Chn',
 '2018/CCI/CCI1104F.Chn',
 '2018/CCI/CCI1104I.Chn',
 '2018/CCI/CCI1403F.Chn',
 '2018/CCI/CCI1403I.Chn',
 '2018/CCI/CCI1508I.Chn',
 '2018/CCI/CCI1605.Chn',
 '2018/CCI/CCI1804F.Chn',
 '2018/CCI/CCI1804I.Chn',
 '2018/CCI/CCI1807.Chn',
 '2018/CCI/CCI2004F-1.Chn',
 '2018/CCI/CCI2004F-2.Chn',
 '2018/CCI/CCI2102F.Chn',
 '2018/CCI/CCI2103F.Chn',
 '2018/CCI/CCI2103I.Chn',
 '2018/CCI/CCI2304F.Chn',
 '2018/CCI/CCI2802F.Chn',
 '2018/CCI/CCI2802I.Chn',
 '2018/CCI/PCI0905 F18.Chn',
 '2018/CCI/PCI1403I.Chn',
 '2018/CRF/CRF0103.Chn',
 '2018/CRF/CRF0111.Chn',
 '2018/CRF/CRF0208.Chn',
 '2018/CRF/CRF0305.Chn',
 '2018/CRF/CRF0410.Chn',
 '2018/CRF/CRF0504.Chn',
 '2018/CRF/CRF0507.Chn',
 '2018/CRF/CRF0609.Chn',
 '2018/CRF/CRF0706.Chn',
 '2018/CRF/CRF0803.Chn',
 '2018/CRF/CR

In [7]:
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [8]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=1017, nomearq = almera22/user2023/SI2022/SI06322.Chn
Final choices:
spectra_path: C:\Users\mmaduar\Nextcloud\Genie_Transfer
a_spec_name: C:\Users\mmaduar\Nextcloud\Genie_Transfer\almera22\user2023\SI2022\SI06322.Chn
reduced_f_name: almera22/user2023/SI2022/SI06322.Chn


(WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'almera22/user2023/SI2022/SI06322.Chn')

In [9]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

energy  unc_en  intensity    unc_i start_level_energy  \
24na  0     472.2023  0.0014   99.90000    0.001           472.2071   
      1     996.8300     0.1    0.00210   0.0002            5235.21   
      2    1368.6250   0.005   99.99400    0.002           1368.667   
      3    2754.0080   0.011   99.86700     0.01           4122.844   
      4    2871.0000       1    0.00025  0.00004            4238.38   
...              ...     ...        ...      ...                ...   
228Ac 267    89.9540            2.01000     0.19                      
      268    93.3470            3.20000      0.3                      
      269   105.5660            1.16000     0.11                      
      270   106.8940            1.54000     0.15                      
      271   108.5800            0.39000     0.04                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      1            4238.38                               NaN                    
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
      4           1368.667         M1+E2          -23      9                    
...                    ...           ...          ...    ...              ...   
228Ac 267                                                                       
      268                                                                       
      269                                                                       
      270                                                                       
      271                                                                       

           ... unc_d         q  unc_q d_z  d_n  d_symbol  \
24na  0    ...                    NaN  11   13        Na   
      1    ...        5515.677   21.0  12   12        Mg   
      2    ...        5515.677   21.0  12   12        Mg   
      3    ...        5515.677   21.0  12   12        Mg   
      4    ...        5515.677   21.0  12   12        Mg   
...        ...   ...       ...    ...  ..  ...       ...   
228Ac 267  ...          2123.8   26.0  90  138        Th   
      268  ...          2123.8   26.0  90  138        Th   
      269  ...          2123.8   26.0  90  138        Th   
      270  ...          2123.8   26.0  90  138        Th   
      271  ...          2123.8   26.0  90  138        Th   

          ensdf_publication_cut-off  \
24na  0                 31-Mar-2022   
      1                 31-Mar-2022   
      2                 31-Mar-2022   
      3                 31-Mar-2022   
      4                 31-Mar-2022   
...                             ...   
228Ac 267               31-Dec-2012   
      268               31-Dec-2012   
      269               31-Dec-2012   
      270               31-Dec-2012   
      271               31-Dec-2012   

                                            ensdf_authors  Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      1    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
      4    M. Shamsuzzoha Basunia and  Anagha Chakraborty       2023-06-15   
...                                                   ...              ...   
228Ac 267                              KHALIFEH ABUSALEEM       2023-06-15   
      268                              KHALIFEH ABUSALEEM       2023-06-15   
      269                              KHALIFEH ABUSALEEM       2023-06-15   
      270                              KHALIFEH ABUSALEEM       2023-06-15   
      271                              KHALIFEH ABUSALEEM       2023-06-15   

          nuclide_name  
24na  0           24na  
      1    

In [10]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
nucl_iear1_df

energy  unc_en  intensity    unc_i start_level_energy  \
24na  0     472.2023  0.0014   99.90000    0.001           472.2071   
      1     996.8300     0.1    0.00210   0.0002            5235.21   
      2    1368.6250   0.005   99.99400    0.002           1368.667   
      3    2754.0080   0.011   99.86700     0.01           4122.844   
      4    2871.0000       1    0.00025  0.00004            4238.38   
...              ...     ...        ...      ...                ...   
228Ac 267    89.9540            2.01000     0.19                      
      268    93.3470            3.20000      0.3                      
      269   105.5660            1.16000     0.11                      
      270   106.8940            1.54000     0.15                      
      271   108.5800            0.39000     0.04                      

          end_level_energy multipolarity mixing_ratio unc_mr conversion_coeff  \
24na  0                  0          [M3]                 NaN          4.69E-4   
      1            4238.38                               NaN                    
      2                  0            E2                 NaN           1.3E-5   
      3           1368.667            E2                 NaN                    
      4           1368.667         M1+E2          -23      9                    
...                    ...           ...          ...    ...              ...   
228Ac 267                                                                       
      268                                                                       
      269                                                                       
      270                                                                       
      271                                                                       

           ...         q  unc_q  d_z  d_n d_symbol  ensdf_publication_cut-off  \
24na  0    ...              NaN   11   13       Na                31-Mar-2022   
      1    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      2    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      3    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
      4    ...  5515.677   21.0   12   12       Mg                31-Mar-2022   
...        ...       ...    ...  ...  ...      ...                        ...   
228Ac 267  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      268  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      269  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      270  ...    2123.8   26.0   90  138       Th                31-Dec-2012   
      271  ...    2123.8   26.0   90  138       Th                31-Dec-2012   

                                            ensdf_authors Extraction_date  \
24na  0    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      1    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      2    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      3    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
      4    M. Shamsuzzoha Basunia and  Anagha Chakraborty      2023-06-15   
...                                                   ...             ...   
228Ac 267                              KHALIFEH ABUSALEEM      2023-06-15   
      268                              KHALIFEH ABUSALEEM      2023-06-15   
      269                              KHALIFEH ABUSALEEM      2023-06-15   
      270                              KHALIFEH ABUSALEEM      2023-06-15   
      271                              KHALIFEH ABUSALEEM      2023-06-15   

           nuclide_name is_to_consider  
24na  0            24na           True  
      1            24na           True  
      2            24na           True  
      3            24na           True  
      4            24na           True  
...                 ...            ...  
228

In [11]:
a_spec = Spec(f_name, reduced_f_name)

In [12]:
dir(a_spec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'channel_energy_calib',
 'curr_h_win',
 'det_descr',
 'f_name',
 'final_composed_baseline',
 'generate_pandas_dataframe',
 'lv_time',
 'net_spec_ser_an',
 'origin_spec_ser_an',
 'pkl_file',
 'reduced_f_name',
 'rl_time',
 'sam_descr',
 'source_datetime',
 'spec_io',
 'start_datetime',
 'sufx',
 'total_analysis']

In [13]:
orig_ser = a_spec.origin_spec_ser_an
vars(orig_ser)

{'mix_regions': None,
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0, 0, 0, ..., 3, 1, 2]),
 'nzero': array([False, False, False, ...,  True,  True,  True]),
 'n_ch': 4096,
 'final_baseline': None,
 'x_s': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
        4.095e+03]),
 'chans_nzero': array([  26.,   27.,   28., ..., 4093., 4094., 4095.]),
 'counts_nzero': array([ 11, 221, 380, ...,   3,   1,   2]),
 'unc_y': array([ 3.31662479, 14.86606875, 19.49358869, ...,  1.73205081,
         1.        ,  1.41421356]),
 'given_variance': array([0, 0, 0, ..., 3, 1, 2]),
 'y_smoothed': array([0, 0, 0, ..., 3, 1, 2]),
 'fft_s': None,
 'is_reg': array([False, False, False, ..., False, False, False]),
 'xs_all_mplets': [],
 'ys_all_mplets': [],
 'ys_all_steps': [],
 'chans_in_multiplets_list': [],
 'calculated_step_counts': [],
 'xs_bl_in_reg': array([], dtype=float64),
 'ys_bl_in_reg': array([], dtype=float64),
 'xs_bl_out_reg': array([], dtype=float64),
 'ys_b

In [14]:
a_spec.total_analysis(gener_dataframe=True)
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

Starting Spec.total_analysis...
Finish Spec.total_analysis!


{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1cf481e6520>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1cf481e6fd0>,
 'f_name': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'reduced_f_name': 'almera22/user2023/SI2022/SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'pkl_file': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1cf47e1eeb0>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x1cf481e68b0>}

In [15]:
vars(orig_ser)

{'mix_regions': array([[  24,   57],
        [ 114,  180],
        [ 236,  267],
        [ 278,  285],
        [ 328,  341],
        [ 352,  363],
        [ 375,  386],
        [ 393,  400],
        [ 443,  450],
        [ 458,  465],
        [ 477,  500],
        [ 524,  535],
        [ 537,  544],
        [ 552,  567],
        [ 576,  589],
        [ 596,  607],
        [ 612,  636],
        [ 669,  674],
        [ 678,  685],
        [ 711,  720],
        [ 721,  745],
        [ 759,  768],
        [ 800,  807],
        [ 812,  827],
        [ 834,  851],
        [ 926,  951],
        [ 954,  984],
        [ 989, 1018],
        [1047, 1054],
        [1076, 1089],
        [1102, 1138],
        [1155, 1177],
        [1182, 1202],
        [1213, 1232],
        [1249, 1266],
        [1275, 1294],
        [1298, 1317],
        [1326, 1350],
        [1366, 1379],
        [1406, 1425],
        [1447, 1462],
        [1491, 1504],
        [1534, 1547],
        [1580, 1598],
        [1637, 16

In [16]:
net_ser = a_spec.net_spec_ser_an
vars(net_ser)

{'mix_regions': array([[  25,   57],
        [ 122,  134],
        [ 139,  150],
        [ 231,  240],
        [ 393,  400],
        [ 479,  498],
        [ 525,  534],
        [ 577,  588],
        [ 712,  719],
        [ 834,  851],
        [ 955,  966],
        [ 998, 1009],
        [1076, 1089],
        [1108, 1123],
        [1125, 1136],
        [1156, 1169],
        [1193, 1200],
        [1250, 1265],
        [1326, 1341],
        [1367, 1378],
        [1448, 1461],
        [1492, 1503],
        [1534, 1547],
        [1580, 1598],
        [1827, 1848],
        [1919, 1934],
        [2027, 2040],
        [2180, 2195],
        [2264, 2281],
        [2390, 2407],
        [2465, 2476],
        [2606, 2623],
        [2832, 2843],
        [2888, 2903],
        [3025, 3036],
        [3438, 3463],
        [3607, 3624],
        [4001, 4028]], dtype=int64),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 2.0245828 , 0.        ,
       

In [17]:
print(a_spec.pkl_file)

C:\Users\mmaduar\Nextcloud\Genie_Transfer\almera22\user2023\SI2022\SI06322.pkl


In [18]:
a_spec.pkl_file

WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.pkl')

In [19]:
# Só para verificar se o pkl_file pode ser recuperado:
read_analysis_spec = pd.read_pickle(a_spec.pkl_file)

In [20]:
read_orig = read_analysis_spec['origin_spec_ser_an'][0]
orig_ser = read_orig
vars(orig_ser)

{'mix_regions': array([[  24,   57],
        [ 114,  180],
        [ 236,  267],
        [ 278,  285],
        [ 328,  341],
        [ 352,  363],
        [ 375,  386],
        [ 393,  400],
        [ 443,  450],
        [ 458,  465],
        [ 477,  500],
        [ 524,  535],
        [ 537,  544],
        [ 552,  567],
        [ 576,  589],
        [ 596,  607],
        [ 612,  636],
        [ 669,  674],
        [ 678,  685],
        [ 711,  720],
        [ 721,  745],
        [ 759,  768],
        [ 800,  807],
        [ 812,  827],
        [ 834,  851],
        [ 926,  951],
        [ 954,  984],
        [ 989, 1018],
        [1047, 1054],
        [1076, 1089],
        [1102, 1138],
        [1155, 1177],
        [1182, 1202],
        [1213, 1232],
        [1249, 1266],
        [1275, 1294],
        [1298, 1317],
        [1326, 1350],
        [1366, 1379],
        [1406, 1425],
        [1447, 1462],
        [1491, 1504],
        [1534, 1547],
        [1580, 1598],
        [1637, 16

In [21]:
read_net = read_analysis_spec['net_spec_ser_an'][0]
net_ser = read_net
vars(net_ser)

{'mix_regions': array([[  25,   57],
        [ 122,  134],
        [ 139,  150],
        [ 231,  240],
        [ 393,  400],
        [ 479,  498],
        [ 525,  534],
        [ 577,  588],
        [ 712,  719],
        [ 834,  851],
        [ 955,  966],
        [ 998, 1009],
        [1076, 1089],
        [1108, 1123],
        [1125, 1136],
        [1156, 1169],
        [1193, 1200],
        [1250, 1265],
        [1326, 1341],
        [1367, 1378],
        [1448, 1461],
        [1492, 1503],
        [1534, 1547],
        [1580, 1598],
        [1827, 1848],
        [1919, 1934],
        [2027, 2040],
        [2180, 2195],
        [2264, 2281],
        [2390, 2407],
        [2465, 2476],
        [2606, 2623],
        [2832, 2843],
        [2888, 2903],
        [3025, 3036],
        [3438, 3463],
        [3607, 3624],
        [4001, 4028]], dtype=int64),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 2.0245828 , 0.        ,
       

In [22]:
# 2023-May-26
# Ok! Espectro gravado, depois lido, então vamos prosseguir
# com a identificação dos nuclídeos.
# Agora, posso ler e analisar um espectro, gravá-lo e depois, em outro momento,
# ler o pkl com a análise.

In [23]:
x_nz = orig_ser.chans_nzero
y_nz = orig_ser.counts_nzero

In [24]:
chans = net_ser.x_s
ys_net_counts = net_ser.y_s
peaks_net = net_ser.pk_parms.peaks
peaks_orig = orig_ser.pk_parms.peaks
counts = orig_ser.y_s
chans_nzero = orig_ser.chans_nzero
counts_nzero = orig_ser.counts_nzero
unc_y = orig_ser.unc_y
eval_y = orig_ser.y_smoothed
eval_bl = orig_ser.eval_baseline
fin_bl = orig_ser.final_baseline
inis = orig_ser.pk_parms.propts['left_bases']
fins = orig_ser.pk_parms.propts['right_bases']

In [25]:
# 2023-Jun-15
# CRUCIAL step: take the dict net_ser.pk_parms' keys/values and organize them as a pd.Dataframe
vars_pkprms = vars(net_ser.pk_parms)
keys_to_get = ['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances']
prep_for_dict = [(key, vars_pkprms[key]) for key in keys_to_get]
pks_dict = dict(prep_for_dict)
peaks_df = pd.DataFrame.from_dict(pks_dict)
peaks_df

,peaks,fwhm_centr,rough_sums,centroids,variances
0,31,31.015664,1623.153269,31.188541,10243.062773
1,42,43.800501,1430.111502,44.293327,21540.729150
2,125,124.694475,1631.001467,125.249005,6686.998533
3,128,128.281527,2933.284105,127.659903,10096.715895
4,144,144.469761,1383.667055,144.548235,7786.332945
5,235,235.390313,332.094692,235.142846,7906.898565
6,396,396.020142,390.814605,396.263267,5348.731117
7,488,487.820179,533.344351,488.149398,10970.082038
8,529,529.359782,297.023953,529.165375,4995.087932
9,582,581.614266,484.212268,581.772500,5194.024894


In [26]:
pr_pk = dict([('pk_hei',net_ser.pk_parms.propts['peak_heights']),
              ('lf_thr',net_ser.pk_parms.propts['left_thresholds']),
              ('rg_thr',net_ser.pk_parms.propts['right_thresholds']),
              ('promns',net_ser.pk_parms.propts['prominences']),
              ('lf_bas',net_ser.pk_parms.propts['left_bases']),
              ('rg_bas',net_ser.pk_parms.propts['right_bases']),
              ('widths',net_ser.pk_parms.propts['widths']),
              ('wi_hei',net_ser.pk_parms.propts['width_heights']),
              ('lf_ips',net_ser.pk_parms.propts['left_ips']),
              ('rg_ips',net_ser.pk_parms.propts['right_ips'])])

In [27]:
teste_df = pd.DataFrame(pr_pk)
teste_df

,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,552.373258,157.062347,216.573947,552.373258,28,36,2.826510,276.186629,29.602409,32.428919
1,178.084376,12.920504,42.207336,178.084376,36,56,6.829875,89.042188,40.385563,47.215438
2,449.256447,71.048679,194.556970,194.556970,119,126,1.611050,351.977962,123.888950,125.500000
3,890.961140,484.133295,239.382450,890.961140,119,134,2.403376,445.480570,127.079839,129.483215
4,520.177021,385.848439,76.998316,520.177021,140,153,2.287660,260.088510,143.325931,145.613591
5,132.552658,91.285233,0.699236,132.552658,224,238,2.232697,66.276329,234.273964,236.506662
6,216.078312,156.591931,147.953356,216.078312,394,404,1.420165,108.039156,395.310059,396.730224
7,99.343812,45.262762,5.755925,99.343812,482,492,4.616647,49.671906,485.511856,490.128502
8,113.048598,74.117583,42.991935,113.048598,526,532,2.244825,56.524299,528.237370,530.482195
9,167.577046,11.007271,92.194617,167.577046,579,585,2.589113,83.788523,580.319710,582.908822


In [64]:
big_df = pd.concat([peaks_df, teste_df],axis=1)
big_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,31,31.015664,1623.153269,31.188541,10243.062773,552.373258,157.062347,216.573947,552.373258,28,36,2.826510,276.186629,29.602409,32.428919
1,42,43.800501,1430.111502,44.293327,21540.729150,178.084376,12.920504,42.207336,178.084376,36,56,6.829875,89.042188,40.385563,47.215438
2,125,124.694475,1631.001467,125.249005,6686.998533,449.256447,71.048679,194.556970,194.556970,119,126,1.611050,351.977962,123.888950,125.500000
3,128,128.281527,2933.284105,127.659903,10096.715895,890.961140,484.133295,239.382450,890.961140,119,134,2.403376,445.480570,127.079839,129.483215
4,144,144.469761,1383.667055,144.548235,7786.332945,520.177021,385.848439,76.998316,520.177021,140,153,2.287660,260.088510,143.325931,145.613591
5,235,235.390313,332.094692,235.142846,7906.898565,132.552658,91.285233,0.699236,132.552658,224,238,2.232697,66.276329,234.273964,236.506662
6,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,404,1.420165,108.039156,395.310059,396.730224
7,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,492,4.616647,49.671906,485.511856,490.128502
8,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,532,2.244825,56.524299,528.237370,530.482195
9,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,585,2.589113,83.788523,580.319710,582.908822


In [29]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  '# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:\n# a_pattern = \'Si/SI2018/SI11318.Chn\'\n# 2022-nov-16: outros espectros:\n# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"\n# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"\n\n# a_pattern = "Eso_non_existe.Chn"\n\n# 2022-Dez-21 Buscando outro espectro\n# a_pattern = \'CCI1603-I\'\n# 2022-Dez-28 Este é um espectro mais recente:\n# a_pattern = \'SI09722.Chn\'\n# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:\n# a_pattern = \'SI06322.Chn\'',
  'import numpy as np\nfrom numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada\nfrom sklearn import linear_model\n\nimport pandas as pd\

In [85]:
def spec_engy_identif(nucl_lib_df, peaks_net_kev_df, func_en, en_toler=2.5, min_intensity = 1.0):
    # Library-based spectrum energy/channel recalibration
    # TODO: implement option to update self.channel_energy_calib

    """Perform a library-based spectrum energy/channel recalibration
    :param func_en: energy function
    :type callable
    :param initial_energy: column specifying the base energy values to consider
    :type initial_energy: string
    :param peaks_net_kev_df: single col 'energy_detected' w/ peaks' energies detected
    :type peaks_net_kev_df: pd.Dataframe
    :param nucl_lib_df: nuclide library based on IAEA format as a Pandas dataframe
    :type nucl_lib_df: pd.Dataframe
    :param en_toler: energy difference accetable to take a detected peak as possibly identified
        (default is 2.5)
    :type en_toler: float
    :param min_intensity: minimum yield percentage to accept a peak
        (default is 1.0)
    :type min_intensity: float
    :returns: a pd.Dataframe with each row as an identified peak with parameters
    :rtype: pd.Dataframe
    """

    left = peaks_net_kev_df
    left['engy_pk_det'] = func_en(left.centroids)
    right = nucl_lib_df
    cross_df = pd.merge(left, right, how='cross')
    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
    df1 = pd.DataFrame(cross_df.loc [
        (cross_df.intensity > min_intensity) & (np.abs(cross_df.delta_en) < en_toler) & cross_df.is_to_consider
        ])
    # [
    #     ['nuclide_name', 'energy', 'energy_detected', 'delta_en',
    #      'intensity', 'unc_i', 'half_life', 'decay', 'decay_%']
    # ]
    X_energy = np.array(df1.energy).reshape(-1,1)
    y_delta_en = np.array(df1.delta_en)
    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X_energy, y_delta_en)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    df1["recalib_engy_ransac"] = df1.engy_pk_det - ransac.predict(X_energy)
    # 2023-Jun-16 PAREI AQUI
    return df1


In [86]:
spec_engy_identif(nucl_iear1_df, big_df, a_spec.channel_energy_calib.get_energy)

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date,nuclide_name,is_to_consider,delta_en,recalib_engy_ransac
7375,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,83,129,Bi,1-Aug-2020,K. Auranen and E.A. McCutchan,2023-06-15,212Pb,True,1.510348,238.935222
7424,396,396.020142,390.814605,396.263267,5348.731117,216.078312,156.591931,147.953356,216.078312,394,...,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,-1.852652,238.933194
8569,488,487.820179,533.344351,488.149398,10970.082038,99.343812,45.262762,5.755925,99.343812,482,...,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,1.064482,295.047223
9148,529,529.359782,297.023953,529.165375,4995.087932,113.048598,74.117583,42.991935,113.048598,526,...,23,28,V,1-Mar-2016,Wang Jimin and Huang Xiaolong,2023-06-15,51cr,True,1.268500,320.094648
10857,582,581.614266,484.212268,581.772500,5194.024894,167.577046,11.007271,92.194617,167.577046,579,...,83,131,Bi,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Pb,True,1.563976,352.220515
11732,715,715.115733,139.405831,715.042172,2414.594169,71.516254,49.552176,37.518001,71.516254,712,...,58,82,Ce,20-Nov-2018,N. NICA,2023-06-15,140la,True,2.436124,433.605074
14191,960,960.262753,405.946228,960.212406,2882.701328,151.182895,73.438463,26.600099,151.182895,951,...,82,126,Pb,1-Jun-2007,M. J. MARTIN,2023-06-15,208Tl,True,1.551030,583.323091
14935,1003,1002.991297,698.096867,1002.890168,2736.208109,276.941300,121.510938,121.636425,276.941300,983,...,45,58,Rh,1-Mar-2009,D. DE FRENNE,2023-06-15,103ru,True,0.482865,609.384554
15491,1003,1002.991297,698.096867,1002.890168,2736.208109,276.941300,121.510938,121.636425,276.941300,983,...,84,130,Po,1-May-2021,Shaofei Zhu and E. A. McCutchan,2023-06-15,214Bi,True,1.494865,609.385164
16131,1082,1082.174080,1959.424012,1082.105291,4312.575988,662.363124,273.496198,118.578263,662.363124,1077,...,48,62,Cd,1-Aug-2011,G. G~URDAL and F.G. KONDEV,2023-06-15,110ag,True,1.719502,657.759742


In [ ]:
f hjrdft jrdjhrstjh rsthj s

In [ ]:
vars(a_spec.channel_energy_calib)

In [54]:
get_energy = a_spec.channel_energy_calib.get_energy

In [55]:
get_energy(1000)

609.0498553514481

In [ ]:
a_spec.pkl_file

In [ ]:
# 2023-May-24
# Implementing:
#  - nuclide labels
# https://plotly.com/python/text-and-annotations/
#  - and energy search in pandas
# https://stackoverflow.com/questions/63529555/get-list-of-occurrences-using-pandas

In [ ]:
# counts_nzero_cps = counts_nzero / a_spec.lv_time
# unc_y_cps = unc_y / a_spec.lv_time
# fin_bl_cps = fin_bl / a_spec.lv_time
# counts_cps = counts / a_spec.lv_time
# en_kev_nzero = get_energy(chans_nzero)
# en_kev = get_energy(chans)
# peaks_net_kev = get_energy(peaks_net)

In [ ]:
help(spec_engy_identif)

In [ ]:
# big_df = identified_peaks_df

aç~]sldgfk~çlaskg~çl

# AQUI: fazer o spec_engy_identif completo

In [ ]:
identified_peaks_df = spec_engy_identif(nucl_iear1_df, big_df, 'energy_detected')
identified_peaks_df

In [ ]:
df1 = identified_peaks_df

In [ ]:
X_energy = np.array(df1.energy).reshape(-1,1)
X_energy

In [ ]:
y_delta_en = np.array(df1.delta_en)
y_delta_en

In [ ]:
# Fit line using all data
X = X_energy
y = y_delta_en
lr = linear_model.LinearRegression()
lr.fit(X, y)
lr.predict(X)

In [ ]:
# Robustly fit linear model with RANSAC algorithm
ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
ransac.predict(X)

In [ ]:
df1["recalib_engy_linear"] = df1.energy_detected - lr.predict(X)
df1["recalib_engy_ransac"] = df1.energy_detected - ransac.predict(X)
# fit() .pr pnuclide_name', 'energy_detected', 'energy', 'delta_en']]

In [ ]:
df1

In [ ]:
big_df = pd.DataFrame(identified_peaks_df)
big_df

In [ ]:
# 2023-Jun-15
# PAREI AQUI
# Descobri um jeito mais eficaz de fazer a recalibração automática, usando um único Dataframe.

In [ ]:
big_df_2 = spec_engy_identif(nucl_iear1_df, big_df, 'recalib_engy_ransac')
# big_df_2 = spec_engy_identif(nucl_iear1_df, big_df, 'energy_detected')
big_df_2

In [ ]:
# Sample size (L, kg, g etc.)
samp_size = 0.1

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [ ]:
# 2023-Jun-14
# Loading calibration pkl file:
calib_pkl_name = 'f100_gmx_2021.pkl'
calib_df = pd.read_pickle(calib_pkl_name)
calib_df

In [ ]:
calib_df.effic_func[3]

In [ ]:
# 2023-Jun-14 PAREI AQUI
# Ver exatamente o que são esses elementos de Polynomial

In [ ]:
p_eff = calib_df.effic_func[3]
p_eff

In [ ]:
p_eff[1]

In [ ]:
p_eff[1](12.3)

In [ ]:
mt_en['uncorr_activ'] = mt_en['rough_sums'] /\
                        (p_eff(mt_en['Engy']) * 1e-4 * mt_en['Yie'] *
                         a_spec.lv_time * samp_size)

In [ ]:
mt_en

In [ ]:
# 2023-Mar-22 PAREI AQUI

In [ ]:
xdfgh rxfth rxthj rds

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

In [ ]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

In [ ]:
# https://stackoverflow.com/questions/63529555

In [ ]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [ ]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

In [ ]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

In [ ]:
help(str)